This file includes the analysis conducted on the pornography consumption survey data.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
# Load and clean the dataset
data = pd.read_csv("")  # File path can be added here for confidentiality.
data = data[~(data['Status'] == "Survey Preview")]  # Remove non-participant rows
data = data[data['Consent'] == "I agree to participate in the research"]  # Keep only rows where participants consented

# Drop unnecessary metadata columns
columns_to_drop = ["UserLanguage", "StartDate", "EndDate", "Status", "DistributionChannel", 
                   "Q_RecaptchaScore", "Q37_Click.Count", "Q38_Click.Count", "Q39_Click.Count", 
                   "Q37_Last.Click", "Q38_Last.Click", "Q39_Last.Click"]
data.drop(columns=columns_to_drop, inplace=True)

# Replace empty string entries with NaN for further cleaning
data.replace("", np.nan, inplace=True)


In [ ]:
# Renaming columns
rename_dict = {
    'Q1': 'age',
    'Q2': 'ethnicity',
    'Q3_1': 'religious_affiliation',
    'Q4': 'education_completed',
    'Q5': 'education_region',
    'Q6': 'alcohol_per_week',
    'Q7': 'gender_identity',
    'Q40_1': 'pre_test_feelings',
    'Q37_First.Click': 'control_group_start',
    'Q37_Page.Submit': 'control_group_end',
    'Q38_First.Click': 'info_group_start',
    'Q38_Page.Submit': 'info_group_end',
    'Q39_First.Click': 'pre_test_start',
    'Q39_Page.Submit': 'pre_test_end',
    'Q36_1': 'info_group_feelings',
    'Q10': 'discomfort_scenario_A',
    'Q11': 'discomfort_scenario_B',
    'Q12': 'discomfort_scenario_C',
    'Q13': 'discomfort_scenario_D',
    'Q14': 'discomfort_scenario_E',
    'Q15': 'discomfort_scenario_F',
    'Q16': 'unclear_scenario_G',
    'Q16.1': 'num_partners',
    'Q17_1': 'sexual_orientation',
    'Q17_1.1': 'frequency_of_sex',
    'Q18': 'age_first_exposure',
    'Q19_1': 'incident_report',
    'Q20': 'relationship_status',
    'Q21_1': 'frequency_of_activity_A',
    'Q22_1': 'frequency_of_activity_B',
    'Q23_1': 'activity_A_without_activity_B',
    'Q24_1': 'comfort_discussing_topic_A',
    'Q25_1': 'discomfort_discussing_topic_A',
    'Q26_1': 'never_discussed_activity_A',
    'Q27_1': 'comfort_discussing_activity_B',
    'Q28_1': 'activity_B_with_partner',
    'Q29_1': 'unaware_activity_B_made',
    'Q30_1': 'guilt_feelings',
    'Q31_1': 'appeal_level',
    'Q32_1': 'awareness_of_creation',
    'Q33_1': 'education_about_topic',
    'Q34_1': 'friends_comfortable',
    'Q35_1': 'people_comfortable'
}

# Apply renaming
data.rename(columns=rename_dict, inplace=True)


In the experiment, participants in the control and treatment groups describe their comfort with a variety of sexually-explicit vignettes. Some vignettes describe consensual scenarios, while others are non-consensual or unclear. In the following sections, scenarios are grouped by consent level.

In [ ]:
# Aggregate comfort level in non-consensual scenarios.

# Define a function to plot comfort levels and save to PDF
def plot_comfort_levels(column, title, filename):
    # Count the values for the given column and fill missing categories with 0
    counts = data[column].value_counts().reindex(
        ["Very uncomfortable", "Uncomfortable", "Neither comfortable nor uncomfortable", 
         "Comfortable", "Very comfortable"], fill_value=0)

    # Create the plot
    plt.figure(figsize=(8, 6))
    sns.barplot(x=counts.index, y=counts.values)
    plt.xticks(rotation=45)
    plt.title(title)
    plt.xlabel("Comfort Level")
    plt.ylabel("Number of Responses")
    plt.tight_layout()

    # Save the plot as a PDF file
    pdf_filename = f"{filename}.pdf"
    plt.savefig(pdf_filename)
    print(f"Saved: {pdf_filename}")
    plt.close()

# List of columns, their corresponding titles, and filenames for PDFs
scenarios = [
    ('discomfort_scenario_A', 'Aggregate Comfort Level with Non-Consensual Scenario A', 'non_consent_scenario_A_comfort_level'),
    ('discomfort_scenario_F', 'Aggregate Comfort Level with Non-Consensual Scenario F', 'non_consent_scenario_F_comfort_level'),
    ('discomfort_scenario_E', 'Aggregate Comfort Level with Non-Consensual Scenario E', 'non_consent_scenario_E_comfort_level')
]

# Loop through each scenario, plot, and save as PDF
for column, title, filename in scenarios:
    plot_comfort_levels(column, title, filename)


In [ ]:
# Combined plot of comfort levels in non-consensual scenarios

# Import GridSpec for arranging plots in a grid
from matplotlib.gridspec import GridSpec

# Create a function to plot multiple scenarios in one figure
def combined_plot_scenarios():
    # Create subplots with GridSpec
    fig = plt.figure(figsize=(18, 6))
    grid = GridSpec(1, 3, figure=fig)

    # Plot for Scenario A
    ax1 = fig.add_subplot(grid[0, 0])
    counts_scenario_A = data['discomfort_scenario_A'].value_counts().reindex(
        ["Very uncomfortable", "Uncomfortable", "Neither comfortable nor uncomfortable", 
         "Comfortable", "Very comfortable"], fill_value=0)
    sns.barplot(x=counts_scenario_A.index, y=counts_scenario_A.values, ax=ax1)
    ax1.set_title('a. Scenario A')
    ax1.set_xlabel('Comfort Level')
    ax1.set_ylabel('Number of Responses')
    ax1.set_xticklabels(counts_scenario_A.index, rotation=45)

    # Plot for Scenario F
    ax2 = fig.add_subplot(grid[0, 1])
    counts_scenario_F = data['discomfort_scenario_F'].value_counts().reindex(
        ["Very uncomfortable", "Uncomfortable", "Neither comfortable nor uncomfortable", 
         "Comfortable", "Very comfortable"], fill_value=0)
    sns.barplot(x=counts_scenario_F.index, y=counts_scenario_F.values, ax=ax2)
    ax2.set_title('b. Scenario F')
    ax2.set_xlabel('Comfort Level')
    ax2.set_ylabel('')
    ax2.set_xticklabels(counts_scenario_F.index, rotation=45)

    # Plot for Scenario E
    ax3 = fig.add_subplot(grid[0, 2])
    counts_scenario_E = data['discomfort_scenario_E'].value_counts().reindex(
        ["Very uncomfortable", "Uncomfortable", "Neither comfortable nor uncomfortable", 
         "Comfortable", "Very comfortable"], fill_value=0)
    sns.barplot(x=counts_scenario_E.index, y=counts_scenario_E.values, ax=ax3)
    ax3.set_title('c. Scenario E')
    ax3.set_xlabel('Comfort Level')
    ax3.set_ylabel('')
    ax3.set_xticklabels(counts_scenario_E.index, rotation=45)

    # Add a title to the overall figure
    fig.suptitle("Figure 2: Comfort with Non-Consensual Scenarios", fontsize=16, fontweight='bold')

    # Save the combined plot as a PDF
    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to fit title
    plt.savefig("combined_non_consent_scenarios.pdf")
    print("Saved: combined_non_consent_scenarios.pdf")
    plt.show()

# Call the function to generate and save the combined plot
combined_plot_scenarios()


In [ ]:
# Filter for Cisgender female participants
data_female = data[data['gender_identity'] == 'Cisgender female'].copy()

# Convert sexual_orientation to numeric
data_female['sexual_orientation'] = pd.to_numeric(data_female['sexual_orientation'], errors='coerce')

# Create a lookup dictionary to map comfort levels to numeric values
comfort_lookup = {
    "Very uncomfortable": 1,
    "Uncomfortable": 2,
    "Neither comfortable nor uncomfortable": 3,
    "Comfortable": 4,
    "Very comfortable": 5
}

# Use the lookup table to replace string values with numeric values for specific columns
data_female['discomfort_scenario_F'] = data_female['discomfort_scenario_F'].map(comfort_lookup)
data_female['discomfort_scenario_E'] = data_female['discomfort_scenario_E'].map(comfort_lookup)

# Subset participants into two groups based on sexual orientation
group_homosexual = data_female[data_female['sexual_orientation'] >= 3].copy()
group_heterosexual = data_female[data_female['sexual_orientation'] < 3].copy()

# Drop NaN values in 'discomfort_scenario_F' for both groups
group_homosexual = group_homosexual.dropna(subset=['discomfort_scenario_F'])
group_heterosexual = group_heterosexual.dropna(subset=['discomfort_scenario_F'])

# Display summary statistics for the discomfort in Scenario F
print(group_homosexual['discomfort_scenario_F'].describe())
print(group_heterosexual['discomfort_scenario_F'].describe())

# Calculate the sample mean and standard error for 'discomfort_scenario_F' in the homosexual group
sample_mean_homosexual = group_homosexual['discomfort_scenario_F'].mean()
se_homosexual = group_homosexual['discomfort_scenario_F'].std() / np.sqrt(len(group_homosexual))

print(f"Sample Mean (Homosexual group): {sample_mean_homosexual}")
print(f"Standard Error (Homosexual group): {se_homosexual}")

# Calculate the z-score and p-value for the difference in means between the two groups
mean_heterosexual = group_heterosexual['discomfort_scenario_F'].mean()
z_score = (sample_mean_homosexual - mean_heterosexual) / se_homosexual
p_value = 2 * (1 - stats.norm.cdf(abs(z_score)))

print(f"Z-Score: {z_score}")
print(f"P-Value: {p_value}")


In [ ]:
# Examine rates of non-answering for participants based on sexual orientation

# Filter out rows where Progress is 100%
data_complete = data[data['Progress'] == 100]

# Subset for Cisgender female participants
data_female_complete = data_complete[data_complete['gender_identity'] == 'Cisgender female']

# Subset for homosexual (sexual_orientation >= 3) and heterosexual (sexual_orientation < 3) groups
group_homosexual = data_female_complete[data_female_complete['sexual_orientation'] >= 3]
group_heterosexual = data_female_complete[data_female_complete['sexual_orientation'] < 3]

# List of columns to check for missing values (NaNs)
cols_to_check = ["discomfort_scenario_A", "discomfort_scenario_F", "discomfort_scenario_E", 
                 "discomfort_scenario_B", "discomfort_scenario_C", "discomfort_scenario_D", "unclear_scenario_G"]

# Calculate the proportion of NaN values for each column in both subsets
prop_na_homosexual = group_homosexual[cols_to_check].isna().mean()
prop_na_heterosexual = group_heterosexual[cols_to_check].isna().mean()

# Create a DataFrame to compare the proportions
prop_na_df = pd.DataFrame({
    'category': cols_to_check,
    'homosexual_prop_na': prop_na_homosexual.values,
    'heterosexual_prop_na': prop_na_heterosexual.values
})

# View the proportions
print(prop_na_df)


In [ ]:
#repeat plotting for consensual scenarios.

# Function to create bar plot for each scenario
def plot_comfort_scenario(data, column, title, ax):
    # Create a target order for the comfort levels
    target_order = ["Very uncomfortable", "Uncomfortable", "Neither comfortable nor uncomfortable", 
                    "Comfortable", "Very comfortable"]
    
    # Count the values for each comfort level, filling missing categories with 0
    counts = data[column].value_counts().reindex(target_order, fill_value=0)
    
    # Create the bar plot on the given axis
    sns.barplot(x=counts.index, y=counts.values, ax=ax)
    ax.set_title(title)
    ax.set_xlabel('Comfort Level')
    ax.set_ylabel('Number of Responses')
    ax.set_xticklabels(counts.index, rotation=45)

# Set up the figure and GridSpec for arranging plots
fig = plt.figure(figsize=(18, 6))
grid = GridSpec(1, 3, figure=fig)

# Plot for Scenario Sam (consensual)
ax1 = fig.add_subplot(grid[0, 0])
plot_comfort_scenario(data, 'comfort_scenario_Sam', 'Aggregate Comfort Level with Consensual Scenario (Sam)', ax1)

# Plot for Scenario Dan
ax2 = fig.add_subplot(grid[0, 1])
plot_comfort_scenario(data, 'comfort_scenario_Dan', 'Aggregate Comfort Level with Consensual Scenario (Dan)', ax2)

# Plot for Scenario Rebecca
ax3 = fig.add_subplot(grid[0, 2])
plot_comfort_scenario(data, 'comfort_scenario_Rebecca', 'Aggregate Comfort Level with Consensual Scenario (Rebecca)', ax3)

# Set the overall title for the figure
fig.suptitle("Figure 3: Comfort with Consensual Scenarios", fontsize=16, fontweight='bold')

# Save the combined plot as a PDF
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to fit title
plt.savefig("combined_consent_scenarios.pdf")
plt.show()


In [ ]:
#recreate above plot with small tweaks

# Filter out rows with NaNs
data_filtered_sam = data[~data['comfort_scenario_Sam'].isna()]
data_filtered_dan = data[~data['comfort_scenario_Dan'].isna()]
data_filtered_rebecca = data[~data['comfort_scenario_Rebecca'].isna()]

# Function to create filtered bar plots
def plot_filtered_scenario(data, column, title, ax):
    # Create a target order for the comfort levels
    target_order = ["Very uncomfortable", "Uncomfortable", "Neither comfortable nor uncomfortable", 
                    "Comfortable", "Very comfortable"]
    
    # Count the values for each comfort level, filling missing categories with 0
    counts = data[column].value_counts().reindex(target_order, fill_value=0)
    
    # Create the bar plot on the given axis
    sns.barplot(x=counts.index, y=counts.values, ax=ax, color="black")
    ax.set_title(title)
    ax.set_xticklabels(['VU', 'U', 'N', 'C', 'VC'], rotation=45)
    ax.set_ylabel('Number of Responses')

# Set up the figure and GridSpec for arranging plots
fig = plt.figure(figsize=(18, 6))
grid = GridSpec(1, 3, figure=fig)

# Plot for Scenario Sam
ax1 = fig.add_subplot(grid[0, 0])
plot_filtered_scenario(data_filtered_sam, 'comfort_scenario_Sam', 'a. Sam', ax1)

# Plot for Scenario Dan
ax2 = fig.add_subplot(grid[0, 1])
plot_filtered_scenario(data_filtered_dan, 'comfort_scenario_Dan', 'b. Dan', ax2)
ax2.set_ylabel('')

# Plot for Scenario Rebecca
ax3 = fig.add_subplot(grid[0, 2])
plot_filtered_scenario(data_filtered_rebecca, 'comfort_scenario_Rebecca', 'c. Rebecca', ax3)
ax3.set_ylabel('')

# Set overall layout and title
plt.tight_layout()
fig.suptitle("Comfort with Consensual Scenarios", fontsize=16, fontweight='bold', y=1.02)
plt.subplots_adjust(top=0.88)

# Save the plot as a PDF
plt.savefig("filtered_consent_scenarios.pdf")
plt.show()


In [ ]:
#redo plotting where consent is unclear

# Filter out missing values in the 'unclear_scenario_G' column
data_filtered_max = data[~data['unclear_scenario_G'].isna()]

# Function to create bar plots for specific scenarios
def plot_unclear_scenario(data, column, ax):
    # Create a target order for the comfort levels
    target_order = ["Very uncomfortable", "Uncomfortable", "Neither comfortable nor uncomfortable", 
                    "Comfortable", "Very comfortable"]
    
    # Count the values for each comfort level, filling missing categories with 0
    counts = data[column].value_counts().reindex(target_order, fill_value=0)
    
    # Create the bar plot
    sns.barplot(x=counts.index, y=counts.values, ax=ax, color="black")
    ax.set_xticklabels(['VU', 'U', 'N', 'C', 'VC'], rotation=45)
    ax.set_ylabel('Number of Responses')

# Set up the figure and GridSpec for arranging plots
fig = plt.figure(figsize=(12, 6))
grid = GridSpec(1, 2, figure=fig)

# Plot for unclear scenario
ax1 = fig.add_subplot(grid[0, 0])
plot_unclear_scenario(data_filtered_max, 'unclear_scenario_G', ax1)
ax1.set_title("Unclear Scenario")
ax1.set_xlabel(None)
ax1.set_ylabel("Number of Responses")

# Plot for Scenario Rebecca (from previous work)
ax2 = fig.add_subplot(grid[0, 1])
plot_unclear_scenario(data_filtered_rebecca, 'comfort_scenario_Rebecca', ax2)
ax2.set_title("Rebecca")
ax2.set_ylabel(None)

# Set the overall layout and save the plot
plt.tight_layout()
plt.subplots_adjust(top=0.88)

# Save the combined plot as a PDF
plt.savefig("unclear_and_rebecca_scenarios.pdf")
plt.show()


To this point, the analysis has captured comfort and response patterns without distinguishing between control and treatment groups. Next, we will examine differences across groups to determine the effects of the two treatments.

In [ ]:
# Assign participant group based on the conditions
data['group_number'] = np.nan  # Add a new column 'group_number' initialized with NaN

# Assign values to 'group_number' based on conditions
data['group_number'] = np.where(data['info_group_start'].notna(), 'info', data['group_number'])
data['group_number'] = np.where(data['pre_test_start'].notna(), 'pressure', data['group_number'])

# For any remaining NaN values, assign 'control' group
data['group_number'].fillna('control', inplace=True)

# Verify the assignment
print(data['group_number'].value_counts())

# Convert end and start times to numeric by replacing commas with dots and converting to float
columns_to_convert = ['info_group_end', 'info_group_start', 'pre_test_end', 'pre_test_start']
for column in columns_to_convert:
    data[column] = data[column].str.replace(",", ".").astype(float)

# Verify the conversions
print(data[['info_group_end', 'info_group_start', 'pre_test_end', 'pre_test_start']].dtypes)


In [ ]:
#examine comfort level overall (not by group)

# Dictionary to replace comfort level strings with numeric values
comfort_map = {
    'Very uncomfortable': 1,
    'Uncomfortable': 2,
    'Neither comfortable nor uncomfortable': 3,
    'Comfortable': 4,
    'Very comfortable': 5
}

# List of columns to convert
columns_to_convert = ['non_chris', 'comfort_scenario_Sam', 'comfort_scenario_Dan', 
                      'comfort_scenario_Rebecca', 'non_jes_disc', 'non_beth', 'unclear_scenario_G']

# Replace comfort levels with numeric values across the specified columns
for column in columns_to_convert:
    data[column] = data[column].replace(comfort_map).astype(float)

# Calculate average comfort levels for aggregate, consensual, and non-consensual scenarios
data['agg_mean'] = data[['non_chris', 'comfort_scenario_Sam', 'comfort_scenario_Dan', 
                         'comfort_scenario_Rebecca', 'non_jes_disc', 'non_beth', 
                         'unclear_scenario_G']].mean(axis=1, skipna=True)

data['con_mean'] = data[['comfort_scenario_Sam', 'comfort_scenario_Dan', 'comfort_scenario_Rebecca']].mean(axis=1, skipna=True)

data['non_mean'] = data[['non_chris', 'non_jes_disc', 'non_beth']].mean(axis=1, skipna=True)

# Calculate median comfort levels
data['agg_med'] = data[['non_chris', 'comfort_scenario_Sam', 'comfort_scenario_Dan', 
                        'comfort_scenario_Rebecca', 'non_jes_disc', 'non_beth', 
                        'unclear_scenario_G']].median(axis=1, skipna=True)

data['con_med'] = data[['comfort_scenario_Sam', 'comfort_scenario_Dan', 'comfort_scenario_Rebecca']].median(axis=1, skipna=True)

data['non_med'] = data[['non_chris', 'non_jes_disc', 'non_beth']].median(axis=1, skipna=True)

# Prepare data for plotting the boxplot of average comfort levels
avg_columns = ['agg_mean', 'con_mean', 'non_mean']
avg_df = data[avg_columns]

# Melt the data for easier plotting
data_long = avg_df.melt(var_name='variable', value_name='value')

# Map variable names to more readable labels
data_long['variable'] = data_long['variable'].map({
    'agg_mean': 'Aggregate',
    'con_mean': 'Consensual',
    'non_mean': 'Non-Consensual'
})

# Create the boxplot
plt.figure(figsize=(8, 6))
sns.boxplot(x='variable', y='value', data=data_long)
plt.title("Average Level of Comfort by Scenario Type", fontsize=12, fontweight='bold')
plt.xlabel("Scenario Type", fontsize=10)
plt.ylabel("Average Level of Comfort", fontsize=10)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.grid(False)  # Remove grid lines
plt.tight_layout()

# Save the plot
plt.savefig("average_comfort_boxplot.pdf")
plt.show()

In [ ]:
#look into sample characteristics

# Gender distribution
gender_counts = data['gender_identity'].value_counts()
print("Gender Distribution:\n", gender_counts)

# Race distribution with percentage
race_counts = data['ethnicity'].value_counts(normalize=True) * 100
race_counts = race_counts.sort_values(ascending=False)
print("Race Distribution:\n", race_counts)

# Age distribution: Replacing "24 or older" with 24 and converting to numeric
data['age'].replace("24 or older", 24, inplace=True)
data['age'] = pd.to_numeric(data['age'], errors='coerce')

# Summary statistics for age
mean_age = data['age'].mean()
median_age = data['age'].median()
sd_age = data['age'].std()
min_age = data['age'].min()
max_age = data['age'].max()

print(f"Mean age: {mean_age}")
print(f"Median age: {median_age}")
print(f"Standard deviation of age: {sd_age}")
print(f"Minimum age: {min_age}")
print(f"Maximum age: {max_age}")

# Religious affiliation distribution and summary statistics
data['religious_affiliation'] = pd.to_numeric(data['religious_affiliation'], errors='coerce')

mean_relig = data['religious_affiliation'].mean()
median_relig = data['religious_affiliation'].median()
sd_relig = data['religious_affiliation'].std()
min_relig = data['religious_affiliation'].min()
max_relig = data['religious_affiliation'].max()

print(f"Mean religious score: {mean_relig}")
print(f"Median religious score: {median_relig}")
print(f"Standard deviation of religious score: {sd_relig}")
print(f"Minimum religious score: {min_relig}")
print(f"Maximum religious score: {max_relig}")

# College completion distribution
college_completed_counts = data['education_completed'].value_counts(normalize=True) * 100
print("College Completed Distribution:\n", college_completed_counts)

# College region distribution
college_region_counts = data['education_region'].value_counts(normalize=True) * 100
print("College Region Distribution:\n", college_region_counts)

# Filter data for participants who completed the survey
data_complete = data[data['Progress'] == 100]

# Sexual orientation distribution
sexuality_counts = data_complete['sexual_orientation'].value_counts(normalize=True) * 100
print("Sexual Orientation Distribution:\n", sexuality_counts)

# Distribution of missing values in 'sexual_orientation' column by gender
na_sexuality = data_complete[data_complete['sexual_orientation'].isna()]
na_sexuality_gender_counts = na_sexuality['gender_identity'].value_counts(normalize=True) * 100
print("Missing Sexuality by Gender Distribution:\n", na_sexuality_gender_counts)

# Assault history by gender
assault_female = data[data['gender_identity'] == 'Cisgender female']['incident_report'].value_counts(normalize=True) * 100
assault_male = data[data['gender_identity'] == 'Cisgender male']['incident_report'].value_counts(normalize=True) * 100

print("Assault History (Female):\n", assault_female)
print("Assault History (Male):\n", assault_male)


In [ ]:
#trends in responses re mastrubation frequency

# Subset participants who did not report masturbation frequency
data_no_mast = data[data['freq_masturbation'].isna()]

# Proportion of each gender with no entry in 'freq_masturbation'
prop_na_by_gender = data_no_mast['gender_identity'].value_counts(normalize=True) * 100
print("Proportion of participants with no entry in 'freq_masturbation' by gender:\n", prop_na_by_gender)

# Convert 'freq_masturbation' to numeric
data['freq_masturbation'] = pd.to_numeric(data['freq_masturbation'], errors='coerce')

# Calculate average frequency of masturbation by gender
avg_freq_by_gender = data.groupby('gender_identity')['freq_masturbation'].mean()
print("Average Frequency of Masturbation by Gender:\n", avg_freq_by_gender)

# Subset data by gender
data_female = data[data['gender_identity'] == 'Cisgender female'].copy()
data_male = data[data['gender_identity'] == 'Cisgender male'].copy()

# Remove rows with missing 'freq_masturbation' values
data_male = data_male.dropna(subset=['freq_masturbation'])
data_female = data_female.dropna(subset=['freq_masturbation'])

# Calculate sample mean and standard error for males
sample_mean_male = data_male['freq_masturbation'].mean()
se_male = data_male['freq_masturbation'].std() / np.sqrt(len(data_male))

# Calculate z-score and p-value for comparison with female sample
sample_mean_female = data_female['freq_masturbation'].mean()
z_score = (sample_mean_male - sample_mean_female) / se_male
p_value = 2 * (1 - stats.norm.cdf(abs(z_score)))

print(f"Z-Score: {z_score}, P-Value: {p_value}")

# Create 'in_relationship' column
data['in_relationship'] = np.where(data['relationship_status'] == 'Yes', 1, 
                                   np.where(pd.isna(data['relationship_status']), np.nan, 0))

# Calculate average frequency of masturbation by relationship status and gender
avg_freq_by_rel_gender = data.groupby(['in_relationship', 'gender_identity'])['freq_masturbation'].mean()
print("Average Frequency by Relationship Status and Gender:\n", avg_freq_by_rel_gender)

# Group data by 'freq_porn' and calculate percentages
freq_porn_distribution = data['freq_porn'].value_counts(normalize=True) * 100
print("Porn Frequency Distribution:\n", freq_porn_distribution)

# Remove rows with missing 'freq_porn'
data_male = data_male.dropna(subset=['freq_porn'])
data_female = data_female.dropna(subset=['freq_porn'])

# Calculate sample mean and standard error for 'freq_porn' in males
sample_mean_porn_male = data_male['freq_porn'].mean()
se_porn_male = data_male['freq_porn'].std() / np.sqrt(len(data_male))

# Calculate z-score and p-value for comparison with female sample
sample_mean_porn_female = data_female['freq_porn'].mean()
z_score_porn = (sample_mean_porn_male - sample_mean_porn_female) / se_porn_male
p_value_porn = 2 * (1 - stats.norm.cdf(abs(z_score_porn)))

print(f"Z-Score (Porn): {z_score_porn}, P-Value: {p_value_porn}")

# Convert 'freq_sex' to numeric and calculate correlations
data['freq_sex'] = pd.to_numeric(data['freq_sex'], errors='coerce')

# Male correlation between 'freq_sex' and 'freq_masturbation'
male_corr = data_male[['freq_sex', 'freq_masturbation']].dropna().corr().loc['freq_sex', 'freq_masturbation']
print(f"Male Correlation (freq_sex and freq_masturbation): {male_corr}")

# Female correlation between 'freq_sex' and 'freq_porn'
female_corr = data_female[['freq_sex', 'freq_porn']].dropna().corr().loc['freq_sex', 'freq_porn']
print(f"Female Correlation (freq_sex and freq_porn): {female_corr}")

# Correlation between 'freq_porn' and 'freq_masturbation' across all participants
data_complete_porn = data.dropna(subset=['freq_porn', 'freq_masturbation'])
porn_mast_corr = data_complete_porn[['freq_porn', 'freq_masturbation']].corr().loc['freq_porn', 'freq_masturbation']
print(f"Correlation between 'freq_porn' and 'freq_masturbation': {porn_mast_corr}")

# Analyze 'mast_no_porn' column
data_complete_mast_no_porn = data.dropna(subset=['mast_no_porn'])
data_complete_mast_no_porn['mast_no_porn'] = pd.to_numeric(data_complete_mast_no_porn['mast_no_porn'], errors='coerce')
print(f"Summary of 'mast_no_porn':\n", data_complete_mast_no_porn['mast_no_porn'].describe())


In [ ]:
#examine social variables around sexuality

# Age when participants first viewed adult material
age_viewed_distribution = data['age_first_exposure'].value_counts(normalize=True) * 100
age_viewed_distribution = age_viewed_distribution.sort_values(ascending=False)
print("Age Viewed Distribution:\n", age_viewed_distribution)

# Convert 'speak_sex' to numeric and filter out rows with missing values
data['speak_sex'] = pd.to_numeric(data['speak_sex'], errors='coerce')
data_speak_sex = data.dropna(subset=['speak_sex'])

# Group by 'speak_sex' and calculate percentages
speak_sex_distribution = data_speak_sex['speak_sex'].value_counts(normalize=True) * 100
print("Speak About Sex Distribution:\n", speak_sex_distribution)

# Convert 'speak_porn' to numeric and filter out rows with missing values
data['speak_porn'] = pd.to_numeric(data['speak_porn'], errors='coerce')
data_speak_porn = data.dropna(subset=['speak_porn'])

# Group by 'speak_porn' and calculate percentages
speak_porn_distribution = data_speak_porn['speak_porn'].value_counts(normalize=True) * 100
print("Speak About Porn Distribution:\n", speak_porn_distribution)

# Convert 'porn_partner' to numeric and filter out rows with missing values
data['porn_partner'] = pd.to_numeric(data['porn_partner'], errors='coerce')
data_porn_partner = data.dropna(subset=['porn_partner'])

# Group by 'porn_partner' and calculate percentages
porn_partner_distribution = data_porn_partner['porn_partner'].value_counts(normalize=True) * 100
print("Porn Partner Distribution:\n", porn_partner_distribution)

# Subset data by gender for 'porn_partner'
data_male_partner = data_porn_partner[data_porn_partner['gender_identity'] == 'Cisgender male']
data_female_partner = data_porn_partner[data_porn_partner['gender_identity'] == 'Cisgender female']

print("Summary of Porn Partner (Male):\n", data_male_partner['porn_partner'].describe())
print("Summary of Porn Partner (Female):\n", data_female_partner['porn_partner'].describe())

# Convert 'never_speak_mast' to numeric and filter out rows with missing values
data['never_speak_mast'] = pd.to_numeric(data['never_speak_mast'], errors='coerce')
data_never_speak_mast = data.dropna(subset=['never_speak_mast'])

# Group by 'never_speak_mast' and calculate percentages
never_speak_mast_distribution = data_never_speak_mast['never_speak_mast'].value_counts(normalize=True) * 100
print("Never Speak About Masturbation Distribution:\n", never_speak_mast_distribution)

# Convert 'educ_sex' to numeric and filter out rows with missing values
data['educ_sex'] = pd.to_numeric(data['educ_sex'], errors='coerce')
data_educ_sex = data.dropna(subset=['educ_sex'])

# Group by 'educ_sex' and calculate percentages
educ_sex_distribution = data_educ_sex['educ_sex'].value_counts(normalize=True) * 100
print("Education About Sex Distribution:\n", educ_sex_distribution)

# Subset data by gender for 'educ_sex'
data_male_educ = data_educ_sex[data_educ_sex['gender_identity'] == 'Cisgender male']
data_female_educ = data_educ_sex[data_educ_sex['gender_identity'] == 'Cisgender female']

print("Summary of Education About Sex (Male):\n", data_male_educ['educ_sex'].describe())
print("Summary of Education About Sex (Female):\n", data_female_educ['educ_sex'].describe())

# Calculate sample mean and standard error for males in 'educ_sex'
sample_mean_male_educ = data_male_educ['educ_sex'].mean()
se_male_educ = data_male_educ['educ_sex'].std() / np.sqrt(len(data_male_educ))

# Calculate the z-score and p-value for comparison with female sample
sample_mean_female_educ = data_female_educ['educ_sex'].mean()
z_score_educ = (sample_mean_male_educ - sample_mean_female_educ) / se_male_educ
p_value_educ = 2 * (1 - stats.norm.cdf(abs(z_score_educ)))

print(f"Z-Score (Education About Sex): {z_score_educ}, P-Value: {p_value_educ}")


In [ ]:
#trends in finding pornography appealing

# Convert 'appeals' to numeric and filter out rows with missing values
data['appeals'] = pd.to_numeric(data['appeals'], errors='coerce')
data_appeals = data.dropna(subset=['appeals'])

# Group by 'appeals' and calculate percentages
appeals_distribution = data_appeals['appeals'].value_counts(normalize=True) * 100
print("Appeals Distribution:\n", appeals_distribution.sort_values(ascending=False))

# Subset data by gender for 'appeals'
data_male_appeals = data_appeals[data_appeals['gender_identity'] == 'Cisgender male']
data_female_appeals = data_appeals[data_appeals['gender_identity'] == 'Cisgender female']

# Summary statistics for 'appeals' by gender
print("Summary of Appeals (Male):\n", data_male_appeals['appeals'].describe())
print("Summary of Appeals (Female):\n", data_female_appeals['appeals'].describe())

# Calculate sample mean and standard error for males in 'appeals'
sample_mean_male_appeals = data_male_appeals['appeals'].mean()
se_male_appeals = data_male_appeals['appeals'].std() / np.sqrt(len(data_male_appeals))

# Calculate the z-score and p-value for comparison with female sample
sample_mean_female_appeals = data_female_appeals['appeals'].mean()
z_score_appeals = (sample_mean_male_appeals - sample_mean_female_appeals) / se_male_appeals
p_value_appeals = 2 * (1 - stats.norm.cdf(abs(z_score_appeals)))

print(f"Z-Score (Appeals): {z_score_appeals}, P-Value: {p_value_appeals}")


In [ ]:
#knowledge and attitudes around pornography

# Convert 'dont_know_porn_made' to numeric and filter out rows with missing values
data['dont_know_porn_made'] = pd.to_numeric(data['dont_know_porn_made'], errors='coerce')
data_dont_know = data.dropna(subset=['dont_know_porn_made'])

# Group by 'dont_know_porn_made' and calculate percentages
dont_know_distribution = data_dont_know['dont_know_porn_made'].value_counts(normalize=True) * 100
print("Don't Know Porn Made Distribution:\n", dont_know_distribution)

# Create a histogram
plt.figure(figsize=(8, 6))
sns.histplot(data_dont_know['dont_know_porn_made'], bins=10, kde=False)
plt.title("Histogram of 'Don't Know Porn Made'")
plt.xlabel("Don't Know Porn Made")
plt.ylabel("Count")
plt.show()

# Subset data by gender for 'dont_know_porn_made'
data_male_dont_know = data_dont_know[data_dont_know['gender_identity'] == 'Cisgender male']
data_female_dont_know = data_dont_know[data_dont_know['gender_identity'] == 'Cisgender female']

print("Summary of Don't Know Porn Made (Male):\n", data_male_dont_know['dont_know_porn_made'].describe())
print("Summary of Don't Know Porn Made (Female):\n", data_female_dont_know['dont_know_porn_made'].describe())

# Calculate sample mean and standard error for males
sample_mean_male_dont_know = data_male_dont_know['dont_know_porn_made'].mean()
se_male_dont_know = data_male_dont_know['dont_know_porn_made'].std() / np.sqrt(len(data_male_dont_know))

# Calculate the z-score and p-value for comparison with female sample
sample_mean_female_dont_know = data_female_dont_know['dont_know_porn_made'].mean()
z_score_dont_know = (sample_mean_male_dont_know - sample_mean_female_dont_know) / se_male_dont_know
p_value_dont_know = 2 * (1 - stats.norm.cdf(abs(z_score_dont_know)))

print(f"Z-Score (Don't Know Porn Made): {z_score_dont_know}, P-Value: {p_value_dont_know}")

# Convert 'guilt_porn' to numeric and filter out rows with missing values
data['guilt_feelings'] = pd.to_numeric(data['guilt_feelings'], errors='coerce')
data_guilt = data.dropna(subset=['guilt_feelings'])

# Group by 'guilt_porn' and calculate percentages
guilt_distribution = data_guilt['guilt_feelings'].value_counts(normalize=True) * 100
print("Guilt Feelings Distribution:\n", guilt_distribution)

# Subset data by gender for 'guilt_feelings'
data_male_guilt = data_guilt[data_guilt['gender_identity'] == 'Cisgender male']
data_female_guilt = data_guilt[data_guilt['gender_identity'] == 'Cisgender female']

print("Summary of Guilt Feelings (Male):\n", data_male_guilt['guilt_feelings'].describe())
print("Summary of Guilt Feelings (Female):\n", data_female_guilt['guilt_feelings'].describe())

# Calculate sample mean and standard error for males
sample_mean_male_guilt = data_male_guilt['guilt_feelings'].mean()
se_male_guilt = data_male_guilt['guilt_feelings'].std() / np.sqrt(len(data_male_guilt))

# Calculate the z-score and p-value for comparison with female sample
sample_mean_female_guilt = data_female_guilt['guilt_feelings'].mean()
z_score_guilt = (sample_mean_male_guilt - sample_mean_female_guilt) / se_male_guilt
p_value_guilt = 2 * (1 - stats.norm.cdf(abs(z_score_guilt)))

print(f"Z-Score (Guilt Feelings): {z_score_guilt}, P-Value: {p_value_guilt}")

# Convert 'uncomfy_speak_sex' to numeric and filter out rows with missing values
data['uncomfy_speak_sex'] = pd.to_numeric(data['uncomfy_speak_sex'], errors='coerce')
data_uncomfy_speak_sex = data.dropna(subset=['uncomfy_speak_sex'])

# Group by 'uncomfy_speak_sex' and calculate percentages
uncomfy_speak_sex_distribution = data_uncomfy_speak_sex['uncomfy_speak_sex'].value_counts(normalize=True) * 100
print("Uncomfortable Speaking About Sex Distribution:\n", uncomfy_speak_sex_distribution)

# Convert 'friends_okay' and 'ppl_okay' to numeric and filter out missing values
data['friends_comfortable'] = pd.to_numeric(data['friends_comfortable'], errors='coerce')
data['people_comfortable'] = pd.to_numeric(data['people_comfortable'], errors='coerce')

data_friends_ppl_okay = data.dropna(subset=['friends_comfortable', 'people_comfortable'])

# Group by 'friends_comfortable' and 'people_comfortable' and calculate percentages
friends_distribution = data_friends_ppl_okay['friends_comfortable'].value_counts(normalize=True) * 100
people_distribution = data_friends_ppl_okay['people_comfortable'].value_counts(normalize=True) * 100

print("Friends Comfortable Distribution:\n", friends_distribution)
print("People Comfortable Distribution:\n", people_distribution)

# Correlation between 'friends_comfortable' and 'people_comfortable'
correlation_friends_ppl_okay = data_friends_ppl_okay[['friends_comfortable', 'people_comfortable']].corr().loc['friends_comfortable', 'people_comfortable']
print(f"Correlation between Friends Comfortable and People Comfortable: {correlation_friends_ppl_okay}")


In [ ]:
#look at survey-taking characteristics

# Convert 'Duration_in_seconds' to numeric
data['duration_seconds'] = pd.to_numeric(data['duration_seconds'], errors='coerce')

# Filter the data for participants who finished the survey
data_complete = data[data['survey_completed'] == True]

# Remove outliers based on the 'duration_seconds' column (mean + 3 * std)
mean_val = data_complete['duration_seconds'].mean()
sd_val = data_complete['duration_seconds'].std()
outlier_threshold = mean_val + 3 * sd_val

# Filter data to exclude outliers
data_no_outliers = data[data['duration_seconds'] <= outlier_threshold]

# Summary statistics for 'duration_seconds' after removing outliers
print("Summary of Duration in Seconds (No Outliers):\n", data_no_outliers['duration_seconds'].describe())

# Analyze time spent by gender (female participants)
data_female_complete = data_complete[data_complete['gender_identity'] == 'Cisgender female']

mean_val_female = data_female_complete['duration_seconds'].mean()
sd_val_female = data_female_complete['duration_seconds'].std()
outlier_threshold_female = mean_val_female + 3 * sd_val_female

# Filter data for females to exclude outliers
data_female_no_outliers = data_female_complete[data_female_complete['duration_seconds'] <= outlier_threshold_female]

# Summary statistics for 'duration_seconds' for females after removing outliers
print("Summary of Duration in Seconds (Female, No Outliers):\n", data_female_no_outliers['duration_seconds'].describe())

# Convert 'Progress' to numeric
data['survey_progress'] = pd.to_numeric(data['survey_progress'], errors='coerce')

# Analyze progress for female participants
data_female_progress = data[data['gender_identity'] == 'Cisgender female']

# Summary of 'survey_progress' for females
print("Summary of Survey Progress (Female):\n", data_female_progress['survey_progress'].describe())


In [ ]:
# List of columns to convert to numeric
columns_to_convert = [
    'guilt_feelings', 'speak_sex', 'freq_porn', 'religious_affiliation', 'people_comfortable', 'friends_comfortable', 
    'incident_report', 'freq_masturbation', 'mast_no_activity', 'uncomfy_speak_sex', 'never_speak_activity', 
    'speak_porn', 'porn_partner', 'dont_know_creation', 'appeals', 'knowledge_creation', 'educ_sex', 
    'friends_comfortable', 'people_comfortable'
]

# Loop through the columns and convert them to numeric
for column in columns_to_convert:
    data[column] = pd.to_numeric(data[column], errors='coerce')

# Clean up categorical variables using mapping where necessary
# College completed
data['education_completed'] = data['education_completed'].map({
    'Less than one year': 0, '1 year': 1, '2 years': 2, '3 years': 3, '4 years': 4, '5+ years': 5
}).astype(float)

# Sexuality to numeric
data['sexual_orientation'] = pd.to_numeric(data['sexual_orientation'], errors='coerce')

# Age cleanup
data['age'] = data['age'].replace({
    '24 or older': 24, '23': 23, '22': 22, '21': 21, '20': 20, '19': 19, '18': 18
}).astype(float)

# Number of partners
data['num_partners'] = data['num_partners'].map({
    '0': 0, '1-2': 1, '3-4': 2, '5-6': 3, '6 or more': 4
}).astype(float)

# Assign numeric values for gender
data['gender_identity'] = data['gender_identity'].map({
    'Cisgender female': 0, 'Cisgender male': 1
}).astype(float)

# Alcohol consumption per week
data['alc_per_week'] = data['alc_per_week'].map({
    '0 times per week': 0, '1-2 times per week': 1, '3-4 times per week': 2, '5-6 times per week': 3, '7 or more times per week': 4
}).astype(float)

# Relationship status
data['relationship_status'] = data['relationship_status'].map({
    'Yes': 1, 'No': 0, 'Maybe': 0
}).astype(float)

# Frequency of sex
data['freq_sex'] = pd.to_numeric(data['freq_sex'], errors='coerce')

# Age first viewed adult material
data['age_first_exposure'] = data['age_first_exposure'].map({
    '9-11': 1, '12-13': 2, '14-15': 3, '16-17': 4, '18+': 5, 'N/A': 0
}).astype(float)

# Verify that columns were correctly converted
print(data.dtypes)



In [ ]:
# Filter rows where 'freq_porn' is not NaN
data_filtered = data.dropna(subset=['freq_porn'])

# Create a binary 'watched_porn' variable based on 'freq_porn'
data_filtered['watched_porn'] = np.where(data_filtered['freq_porn'].isin([0, 1]), 0, 1)

# Create a binary 'race_white' variable based on race
data_filtered['race_white'] = np.where(data_filtered['ethnicity'] == 'White / Caucasian', 1, 0)

# Verify the new columns
print(data_filtered[['watched_porn', 'race_white']].head())


In [ ]:
#verifying htat my results align with the primary study that this project is based on
import statsmodels.api as sm
import numpy as np

# Prepare the independent variables and dependent variable for the logistic regression model
X = data_filtered[['gender_identity', 'age', 'race_white', 'religious_affiliation', 'education_completed', 'sexual_orientation', 'alc_per_week']]
X = sm.add_constant(X)  # Add a constant term for the intercept
y = data_filtered['watched_porn']

# Fit the logistic regression model (equivalent to glm with binomial family in R)
modelA = sm.Logit(y, X).fit()

# Print the summary of the model
print(modelA.summary())

# Extract null deviance and degrees of freedom
null_deviance = modelA.llnull
df_null = modelA.df_model + 1  # Adding 1 for intercept

# Number of observations
n_obs = len(modelA.fittedvalues)
print(f"Null Deviance: {null_deviance}")
print(f"Degrees of Freedom (Null): {df_null}")
print(f"Number of Observations: {n_obs}")


In [ ]:
#replicating model A from cooper and klein:

import numpy as np
import pandas as pd
import statsmodels.api as sm

# Prepare the independent variables and dependent variable for the logistic regression model
X = data_filtered[['gender_identity', 'age', 'race_white', 'religious_affiliation', 'education_completed', 'sexual_orientation', 'alc_per_week']]
X = sm.add_constant(X)  # Add a constant term for the intercept
y = data_filtered['watched_porn']

# Fit the logistic regression model
modelA = sm.Logit(y, X).fit()

# Extract coefficients and standard errors
coefficients = modelA.params
std_errors = modelA.bse

# Calculate odds ratios using exp()
odds_ratios = np.exp(coefficients)

# Calculate standard errors of the odds ratios
odds_ratios_std_errors = odds_ratios * std_errors

# Extract p-values from the model
p_values = modelA.pvalues

# Create a DataFrame to store the results
result = pd.DataFrame({
    'Coefficient': coefficients,
    'Std_Error': std_errors,
    'Odds_Ratio': odds_ratios,
    'Odds_Ratio_Std_Error': odds_ratios_std_errors,
    'P_Value': p_values
})

# Calculate residuals and fitted values
residuals = modelA.resid_response
fitted_values = modelA.fittedvalues

# Calculate chi-squared statistic
chi_squared = np.sum((residuals ** 2) / (fitted_values * (1 - fitted_values)))
df = len(coefficients) - 1
nobs = len(residuals)

# Add chi-squared, degrees of freedom, and number of observations to the results
result['Chi_Squared'] = chi_squared
result['Degrees_Freedom'] = df
result['N_Obs'] = nobs

# Display the result DataFrame
print(result)


In [ ]:
#now model B for keep and klein:

import numpy as np
import pandas as pd
import statsmodels.api as sm

# Prepare the independent variables and dependent variable for the logistic regression model (Model B)
X_B = data_filtered[['gender_identity', 'age', 'race_white', 'religious_affiliation', 'education_completed', 
                     'sexual_orientation', 'alc_per_week', 'num_partners', 'relationship_status', 'freq_sex', 
                     'age_first_exposure', 'freq_masturbation']]
X_B = sm.add_constant(X_B)  # Add a constant term for the intercept
y_B = data_filtered['watched_porn']

# Fit the logistic regression model (Model B)
modelB = sm.Logit(y_B, X_B).fit()

# Print the summary of the model
print(modelB.summary())

# Extract null deviance and degrees of freedom
null_deviance_B = modelB.llnull
df_null_B = modelB.df_model + 1  # Adding 1 for intercept

# Number of observations
n_obs_B = len(modelB.fittedvalues)
print(f"Null Deviance (Model B): {null_deviance_B}")
print(f"Degrees of Freedom (Null, Model B): {df_null_B}")
print(f"Number of Observations (Model B): {n_obs_B}")

# Extract coefficients, standard errors, and p-values
coefficients_B = modelB.params
std_errors_B = modelB.bse
p_values_B = modelB.pvalues

# Calculate odds ratios using exp()
odds_ratios_B = np.exp(coefficients_B)

# Calculate standard errors of the odds ratios
odds_ratios_std_errors_B = odds_ratios_B * std_errors_B

# Create a DataFrame to store the results
result_B = pd.DataFrame({
    'Coefficient': coefficients_B,
    'Std_Error': std_errors_B,
    'Odds_Ratio': odds_ratios_B,
    'Odds_Ratio_Std_Error': odds_ratios_std_errors_B,
    'P_Value': p_values_B
})

# Calculate residuals and fitted values
residuals_B = modelB.resid_response
fitted_values_B = modelB.fittedvalues

# Calculate chi-squared statistic
chi_squared_B = np.sum((residuals_B ** 2) / (fitted_values_B * (1 - fitted_values_B)))
df_B = len(coefficients_B) - 1
nobs_B = len(residuals_B)

# Add chi-squared, degrees of freedom, and number of observations to the results
result_B['Chi_Squared'] = chi_squared_B
result_B['Degrees_Freedom'] = df_B
result_B['N_Obs'] = nobs_B

# Display the result DataFrame
print(result_B)


In [ ]:
#now model C from Cooper and Klein

import numpy as np
import pandas as pd
import statsmodels.api as sm

# Prepare the independent variables and dependent variable for the logistic regression model (Model C)
X_C = data_filtered[['gender_identity', 'age', 'race_white', 'religious_affiliation', 'education_completed', 
                     'sexual_orientation', 'alc_per_week', 'num_partners', 'relationship_status', 'freq_sex', 
                     'age_first_exposure', 'freq_masturbation', 'speak_porn', 'speak_sex', 
                     'friends_comfortable', 'people_comfortable']]
X_C = sm.add_constant(X_C)  # Add a constant term for the intercept
y_C = data_filtered['watched_porn']

# Fit the logistic regression model (Model C)
modelC = sm.Logit(y_C, X_C).fit()

# Print the summary of the model
print(modelC.summary())

# Extract null deviance and degrees of freedom
null_deviance_C = modelC.llnull
df_null_C = modelC.df_model + 1  # Adding 1 for intercept

# Number of observations
n_obs_C = len(modelC.fittedvalues)
print(f"Null Deviance (Model C): {null_deviance_C}")
print(f"Degrees of Freedom (Null, Model C): {df_null_C}")
print(f"Number of Observations (Model C): {n_obs_C}")

# Extract coefficients, standard errors, and p-values
coefficients_C = modelC.params
std_errors_C = modelC.bse
p_values_C = modelC.pvalues

# Calculate odds ratios using exp()
odds_ratios_C = np.exp(coefficients_C)

# Calculate standard errors of the odds ratios
odds_ratios_std_errors_C = odds_ratios_C * std_errors_C

# Create a DataFrame to store the results
result_C = pd.DataFrame({
    'Coefficient': coefficients_C,
    'Std_Error': std_errors_C,
    'Odds_Ratio': odds_ratios_C,
    'Odds_Ratio_Std_Error': odds_ratios_std_errors_C,
    'P_Value': p_values_C
})

# Calculate residuals and fitted values
residuals_C = modelC.resid_response
fitted_values_C = modelC.fittedvalues

# Calculate chi-squared statistic
chi_squared_C = np.sum((residuals_C ** 2) / (fitted_values_C * (1 - fitted_values_C)))
df_C = len(coefficients_C) - 1
nobs_C = len(residuals_C)

# Add chi-squared, degrees of freedom, and number of observations to the results
result_C['Chi_Squared'] = chi_squared_C
result_C['Degrees_Freedom'] = df_C
result_C['N_Obs'] = nobs_C

# Display the result DataFrame
print(result_C)


In [ ]:
#did men often stop taking the survey after reading the vignettes? (sign of discomfort)

# Get the index of the 'unclear_scenario_G' column
unc_max_col = data.columns.get_loc('unclear_scenario_G')

# Select columns from 'unclear_scenario_G' to the end
selected_cols = ['gender_identity'] + data.columns[unc_max_col + 1:].tolist()

# Subset the data frame with the selected columns
data_filtered = data[selected_cols]

# Split the data into male and female subsets
female_df = data_filtered[data_filtered['gender_identity'] == 0]
male_df = data_filtered[data_filtered['gender_identity'] == 1]

# Calculate the number of missing values in each row for males and females
female_na_count = female_df.isna().sum(axis=1)
male_na_count = male_df.isna().sum(axis=1)

# Calculate and print the mean number of missing values for each gender
mean_female_na = female_na_count.mean()
mean_male_na = male_na_count.mean()

print(f"Mean number of missing values (Female): {mean_female_na}")
print(f"Mean number of missing values (Male): {mean_male_na}")

#now same Q, but by group
# Subset the dataset to only include males
males = data_filtered[data_filtered['gender_identity'] == 1]

# Subset the male dataset to only include "info" and "control" groups
male_info_control = males[males['group_number'].isin(['info', 'control'])]

# Create a new column called "num_nas" that counts the number of NAs in each row
male_info_control['num_nas'] = male_info_control.isna().sum(axis=1)

# Calculate the mean number of NAs for males in the "info" group and "control" group
mean_nas_info = male_info_control.loc[male_info_control['group_number'] == 'info', 'num_nas'].mean()
mean_nas_control = male_info_control.loc[male_info_control['group_number'] == 'control', 'num_nas'].mean()

# Print the results
print(f"Mean number of NAs for males in the 'info' group: {mean_nas_info}")
print(f"Mean number of NAs for males in the 'control' group: {mean_nas_control}")


In [ ]:
#look at trends in time spent on conditions - check to see if they actually read it, and separating out effects for those who did read it

# Create new columns for the amount of time spent reading the condition
data['control_time_spent'] = np.nan  # Create a column but leave it empty for now
data['info_time_spent'] = data['info_group_end'] - data['info_group_start']
data['pre_time_spent'] = data['pre_test_end'] - data['pre_test_start']

# Calculate mean of control_time_spent for group "control"
mean_control = data.groupby('group_number')['control_time_spent'].mean().loc['control']

# Calculate mean of info_time_spent for group "info"
mean_info = data.groupby('group_number')['info_time_spent'].mean().loc['info']

# Calculate mean of pre_time_spent for group "pressure"
mean_pressure = data.groupby('group_number')['pre_time_spent'].mean().loc['pressure']

# Filter data to include only female participants
data_female = data[data['gender_identity'] == 0]

# Convert 'info_feels_pos' and 'pre_feels_pos' to numeric
data['info_feels_pos'] = pd.to_numeric(data['info_feels_pos'], errors='coerce')
data['pre_feels_pos'] = pd.to_numeric(data['pre_feels_pos'], errors='coerce')

# Summary statistics for 'info_feels_pos'
print("Summary of 'info_feels_pos':\n", data['info_feels_pos'].describe())

# Remove rows where less than a certain amount of time was spent on specific conditions
data_filtered = data[~((data['group_number'] == 'pressure') & (data['pre_time_spent'] < 30))]
data_filtered = data_filtered[~((data['group_number'] == 'info') & (data['info_time_spent'] < 20))]

# Verify the filtering
print(f"Rows remaining after filtering: {len(data_filtered)}")


In [ ]:
#r did men stop taking the survey after porn vignettes (BY group)

# Get the index of the 'unclear_scenario_G' column
unc_max_col = data.columns.get_loc('unclear_scenario_G')

# Select columns from 'unclear_scenario_G' to the end, including 'gender_identity'
selected_cols = ['gender_identity'] + data.columns[unc_max_col + 1:].tolist()

# Subset the data with the selected columns
data_filtered = data[selected_cols]

# Subset the dataset to only include males (gender == 0)
males = data_filtered[data_filtered['gender_identity'] == 0]

# Subset the male dataset to only include "info" and "control" groups
male_info_control = males[males['group_number'].isin(['info', 'control'])]

# Create a new column called "num_nas" that counts the number of NAs in each row
male_info_control['num_nas'] = male_info_control.isna().sum(axis=1)

# Calculate the mean number of NAs for males in the "info" group and "control" group
mean_nas_info = male_info_control.loc[male_info_control['group_number'] == 'info', 'num_nas'].mean()
mean_nas_control = male_info_control.loc[male_info_control['group_number'] == 'control', 'num_nas'].mean()

# Print the results
print(f"Mean number of NAs for males in the 'info' group: {mean_nas_info}")
print(f"Mean number of NAs for males in the 'control' group: {mean_nas_control}")


In [ ]:
#figures in final report

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols

# Set up the theme for plots
sns.set(style="whitegrid")

# Create new treatment variable and map conditions
data['treatment'] = data['group_number'].replace({'info': 'treatment', 'pressure': 'treatment', 'control': 'control'})

# Melt the data for easy plotting
avgs = ['treatment', 'agg_mean', 'con_mean', 'non_mean', 'unclear_scenario_G']
avg_df = data[avgs]
data_long = avg_df.melt(id_vars='treatment', var_name='variable', value_name='value')

# Update variable names for better readability
data_long['variable'] = data_long['variable'].replace({
    'agg_mean': 'Aggregate', 
    'con_mean': 'Consensual', 
    'non_mean': 'Non-Consensual', 
    'unclear_scenario_G': 'Unclear'
})

# Drop rows with missing values
data_long = data_long.dropna(subset=['value'])

# Summary statistics and standard errors for plotting
data_long_summary = data_long.groupby(['treatment', 'variable']).agg(
    mean_value=('value', 'mean'),
    se=('value', lambda x: np.std(x, ddof=1) / np.sqrt(len(x)))
).reset_index()

# Plotting: Boxplot for average comfort by scenario type, split by treatment
plt.figure(figsize=(10, 6))
sns.boxplot(x='variable', y='value', hue='treatment', data=data_long, palette='muted')
plt.title('Level of Comfort with Scenarios for all Participants')
plt.xlabel('Scenario Type')
plt.ylabel('Average Level of Comfort')
plt.show()

# Bar plot with error bars for mean values
plt.figure(figsize=(10, 6))
sns.barplot(x='variable', y='mean_value', hue='treatment', data=data_long_summary, 
            palette={'info': '#9DC3E6', 'pressure': '#FAC8CD', 'control': '#CCF0E6'}, dodge=True, ci=None)

# Add error bars
for idx, row in data_long_summary.iterrows():
    plt.errorbar(x=idx % len(data_long_summary['variable'].unique()), y=row['mean_value'], 
                 yerr=row['se'], fmt='none', color='black', capsize=5)

plt.title('Average Level of Comfort by Treatment and Scenario Type')
plt.xlabel('Scenario Type')
plt.ylabel('Average Level of Comfort')
plt.show()

# Create plots for male and female participants separately
for gender, gender_name in [(0, 'Female'), (1, 'Male')]:
    gender_data = data[data['gender_identity'] == gender]
    avg_df_gender = gender_data[['treatment', 'agg_mean', 'con_mean', 'non_mean', 'unclear_scenario_G']]
    data_long_gender = avg_df_gender.melt(id_vars='treatment', var_name='variable', value_name='value')
    data_long_gender['variable'] = data_long_gender['variable'].replace({
        'agg_mean': 'Aggregate', 
        'con_mean': 'Consensual', 
        'non_mean': 'Non-Consensual', 
        'unclear_scenario_G': 'Unclear'
    })
    data_long_gender_summary = data_long_gender.groupby(['treatment', 'variable']).agg(
        mean_value=('value', 'mean'),
        se=('value', lambda x: np.std(x, ddof=1) / np.sqrt(len(x)))
    ).reset_index()

    # Bar plot for each gender
    plt.figure(figsize=(10, 6))
    sns.barplot(x='variable', y='mean_value', hue='treatment', data=data_long_gender_summary, dodge=True, 
                palette={'info': '#9DC3E6', 'pressure': '#FAC8CD', 'control': '#CCF0E6'})
    
    # Add error bars
    for idx, row in data_long_gender_summary.iterrows():
        plt.errorbar(x=idx % len(data_long_gender_summary['variable'].unique()), y=row['mean_value'], 
                     yerr=row['se'], fmt='none', color='black', capsize=5)
    
    plt.title(f'Average Level of Comfort by Scenario Type - {gender_name}')
    plt.xlabel('Scenario Type')
    plt.ylabel('Average Level of Comfort')
    plt.show()

# Overall plot for male and female participants for all groups
data_long_summary['treatment'] = data_long_summary['treatment'].replace({0: 'female', 1: 'male'})
data_long_summary = data_long_summary.dropna(subset=['treatment'])

# Plot for male and female participants
plt.figure(figsize=(10, 6))
sns.barplot(x='variable', y='mean_value', hue='treatment', data=data_long_summary, dodge=True, 
            palette={'female': '#FAC8CD', 'male': '#9DC3E6'})
plt.title('Average Level of Comfort by Gender')
plt.xlabel('Scenario Type')
plt.ylabel('Average Level of Comfort')
plt.show()


In [ ]:
#more plots in main file

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Function to prepare data for plotting
def prepare_data(data, group_filter):
    # Subset the data by group (control, info, pressure)
    data_group = data[data['group_number'] == group_filter].copy()

    # Map gender to treatment
    data_group['treatment'] = data_group['gender_identity'].replace({0: 'female', 1: 'male'})

    # Melt the data for easy plotting
    avgs = ['treatment', 'con_mean', 'non_mean', 'unclear_scenario_G']
    avg_df = data_group[avgs]
    data_long = avg_df.melt(id_vars='treatment', var_name='variable', value_name='value')

    # Rename the variable for clarity
    data_long['variable'] = data_long['variable'].replace({
        'con_mean': 'Consensual',
        'non_mean': 'Non-Consensual',
        'unclear_scenario_G': 'Unclear'
    })

    # Group by treatment and variable, calculate mean and standard error
    data_long_summary = data_long.groupby(['treatment', 'variable']).agg(
        mean_value=('value', 'mean'),
        se=('value', lambda x: np.std(x, ddof=1) / np.sqrt(len(x)))
    ).reset_index()

    return data_long_summary

# Prepare data for the "control", "info", and "pressure" groups
data_control = prepare_data(data, 'control')
data_info = prepare_data(data, 'info')
data_pressure = prepare_data(data, 'pressure')

# Function to create bar plots with error bars
def plot_bar(data, title, ax):
    sns.barplot(x='variable', y='mean_value', hue='treatment', data=data, dodge=True, palette={'female': '#FAC8CD', 'male': '#9DC3E6'}, ax=ax)
    
    # Add error bars
    for idx, row in data.iterrows():
        ax.errorbar(x=idx % len(data['variable'].unique()), y=row['mean_value'], 
                     yerr=row['se'], fmt='none', color='black', capsize=5)

    ax.set_title(title)
    ax.set_xlabel('Scenario Type')
    ax.set_ylabel('Mean Comfort')
    ax.tick_params(axis='x', rotation=45)

# Set up the grid for plotting the three groups together
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot for "control" group
plot_bar(data_control, 'a. Control', axes[0])

# Plot for "info" group
plot_bar(data_info, 'b. Information', axes[1])

# Plot for "pressure" group
plot_bar(data_pressure, 'c. Social Pressure', axes[2])

# Adjust layout for better spacing
plt.tight_layout()
plt.show()


In [ ]:
#regression analysis

import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 1. Recode gender and combine cis and trans
data['gender_identity'] = data['gender_identity'].replace({0: 'female', 1: 'male'})

# 2. Linear regression models with interaction terms
# Define a function to run and summarize regression models
def run_regression(formula, data):
    model = ols(formula, data=data).fit()
    return model.summary()

# Model 1: Aggregate mean regression
agg_model = run_regression('agg_mean ~ treatment + guilt_feelings + treatment*guilt_feelings + people_comfortable + friends_comfortable + freq_porn + speak_sex + religious_affiliation + gender_identity + treatment*gender_identity', data)
print(agg_model)

# Model 2: Consensual mean regression
con_model = run_regression('con_mean ~ treatment + guilt_feelings + treatment*guilt_feelings + people_comfortable + friends_comfortable + freq_porn + speak_sex + religious_affiliation + gender_identity + treatment*gender_identity', data)
print(con_model)

# Model 3: Non-consensual mean regression
non_model = run_regression('non_mean ~ treatment + guilt_feelings + treatment*guilt_feelings + people_comfortable + friends_comfortable + freq_porn + speak_sex + religious_affiliation + gender_identity + treatment*gender_identity', data)
print(non_model)

# Model 4: Unclear mean regression
unclear_model = run_regression('unclear_scenario_G ~ treatment + guilt_feelings + treatment*guilt_feelings + people_comfortable + friends_comfortable + freq_porn + speak_sex + religious_affiliation + gender_identity + treatment*gender_identity', data)
print(unclear_model)

# 3. Run regression models for low comfort participants
data_low = data[data['agg_mean'] <= 3]

agg_low_model = run_regression('agg_mean ~ treatment + guilt_feelings + treatment*guilt_feelings + people_comfortable + friends_comfortable + freq_porn + speak_sex + religious_affiliation + gender_identity + treatment*gender_identity', data_low)
print(agg_low_model)

# Run regression models for high comfort participants
data_high = data[data['agg_mean'] >= 3]

agg_high_model = run_regression('agg_mean ~ treatment + guilt_feelings + treatment*guilt_feelings + people_comfortable + friends_comfortable + freq_porn + speak_sex + religious_affiliation + gender_identity + treatment*gender_identity', data_high)
print(agg_high_model)

# 4. Mean comparisons between men and women
# Subset the data based on group_number
data_control = data[data['group_number'] == "control"]
data_info = data[data['group_number'] == "info"]
data_pressure = data[data['group_number'] == "pressure"]

# Calculate mean for agg_mean across groups (control, info, pressure)
mean_control = data_control['agg_mean'].mean()
mean_info = data_info['agg_mean'].mean()
mean_pressure = data_pressure['agg_mean'].mean()

print(f"Mean (Control): {mean_control}")
print(f"Mean (Info): {mean_info}")
print(f"Mean (Pressure): {mean_pressure}")

# Calculate sample mean, standard error, z-score, and p-value for 'info' group
se_info = data_info['agg_mean'].std() / np.sqrt(len(data_info))
z_score_info = (mean_info - mean_control) / se_info
p_value_info = 2 * (1 - stats.norm.cdf(abs(z_score_info)))

print(f"Z-Score (Info vs Control): {z_score_info}, P-Value: {p_value_info}")

# Calculate sample mean, standard error, z-score, and p-value for 'pressure' group
se_pressure = data_pressure['agg_mean'].std() / np.sqrt(len(data_pressure))
z_score_pressure = (mean_pressure - mean_control) / se_pressure
p_value_pressure = 2 * (1 - stats.norm.cdf(abs(z_score_pressure)))

print(f"Z-Score (Pressure vs Control): {z_score_pressure}, P-Value: {p_value_pressure}")


In [ ]:
#testing significance of both treatments for men

import numpy as np
from scipy import stats

# Subset the data for males by group number (control, info, pressure)
data_male_control = data_male[data_male['group_number'] == 'control']
data_male_info = data_male[data_male['group_number'] == 'info']
data_male_pressure = data_male[data_male['group_number'] == 'pressure']

# Filter out rows with missing values for agg_mean
data_male_control_agg = data_male_control.dropna(subset=['agg_mean'])
data_male_info_agg = data_male_info.dropna(subset=['agg_mean'])
data_male_pressure_agg = data_male_pressure.dropna(subset=['agg_mean'])

# Print the means of agg_mean for control, info, and pressure groups
print(f"Mean (Control, agg_mean): {data_male_control_agg['agg_mean'].mean()}")
print(f"Mean (Info, agg_mean): {data_male_info_agg['agg_mean'].mean()}")
print(f"Mean (Pressure, agg_mean): {data_male_pressure_agg['agg_mean'].mean()}")

# Z-test: Compare agg_mean between info and control groups
sample_mean_info = data_male_info_agg['agg_mean'].mean()
se_info = data_male_info_agg['agg_mean'].std() / np.sqrt(len(data_male_info_agg))

z_score_info = (sample_mean_info - data_male_control_agg['agg_mean'].mean()) / se_info
p_value_info = 2 * (1 - stats.norm.cdf(abs(z_score_info)))

print(f"Z-Score (Info vs Control, agg_mean): {z_score_info}, P-Value: {p_value_info}")

# Z-test: Compare agg_mean between pressure and control groups
sample_mean_pressure = data_male_pressure_agg['agg_mean'].mean()
se_pressure = data_male_pressure_agg['agg_mean'].std() / np.sqrt(len(data_male_pressure_agg))

z_score_pressure = (sample_mean_pressure - data_male_control_agg['agg_mean'].mean()) / se_pressure
p_value_pressure = 2 * (1 - stats.norm.cdf(abs(z_score_pressure)))

print(f"Z-Score (Pressure vs Control, agg_mean): {z_score_pressure}, P-Value: {p_value_pressure}")

# Subset the data for con_mean
data_male_control_con = data_male_control.dropna(subset=['con_mean'])
data_male_info_con = data_male_info.dropna(subset=['con_mean'])
data_male_pressure_con = data_male_pressure.dropna(subset=['con_mean'])

# Print the means of con_mean for control, info, and pressure groups
print(f"Mean (Control, con_mean): {data_male_control_con['con_mean'].mean()}")
print(f"Mean (Info, con_mean): {data_male_info_con['con_mean'].mean()}")
print(f"Mean (Pressure, con_mean): {data_male_pressure_con['con_mean'].mean()}")

# Z-test: Compare con_mean between info and control groups
sample_mean_con_info = data_male_info_con['con_mean'].mean()
se_con_info = data_male_info_con['con_mean'].std() / np.sqrt(len(data_male_info_con))

z_score_con_info = (sample_mean_con_info - data_male_control_con['con_mean'].mean()) / se_con_info
p_value_con_info = 2 * (1 - stats.norm.cdf(abs(z_score_con_info)))

print(f"Z-Score (Info vs Control, con_mean): {z_score_con_info}, P-Value: {p_value_con_info}")

# Z-test: Compare con_mean between pressure and control groups
sample_mean_con_pressure = data_male_pressure_con['con_mean'].mean()
se_con_pressure = data_male_pressure_con['con_mean'].std() / np.sqrt(len(data_male_pressure_con))

z_score_con_pressure = (sample_mean_con_pressure - data_male_control_con['con_mean'].mean()) / se_con_pressure
p_value_con_pressure = 2 * (1 - stats.norm.cdf(abs(z_score_con_pressure)))

print(f"Z-Score (Pressure vs Control, con_mean): {z_score_con_pressure}, P-Value: {p_value_con_pressure}")



In [ ]:
#looking at the same stats, but only non-consensual
import numpy as np
from scipy import stats

# Subset the data for males by group number (control, info, pressure)
data_male_control = data_male[data_male['group_number'] == 'control']
data_male_info = data_male[data_male['group_number'] == 'info']
data_male_pressure = data_male[data_male['group_number'] == 'pressure']

# Filter out rows with missing values for non_mean
data_male_control_non_mean = data_male_control.dropna(subset=['non_mean'])
data_male_info_non_mean = data_male_info.dropna(subset=['non_mean'])
data_male_pressure_non_mean = data_male_pressure.dropna(subset=['non_mean'])

# Print the means of non_mean for control, info, and pressure groups
print(f"Mean (Control, non_mean): {data_male_control_non_mean['non_mean'].mean()}")
print(f"Mean (Info, non_mean): {data_male_info_non_mean['non_mean'].mean()}")
print(f"Mean (Pressure, non_mean): {data_male_pressure_non_mean['non_mean'].mean()}")

# Z-test: Compare non_mean between info and control groups
sample_mean_info = data_male_info_non_mean['non_mean'].mean()
se_info = data_male_info_non_mean['non_mean'].std() / np.sqrt(len(data_male_info_non_mean))

z_score_info = (sample_mean_info - data_male_control_non_mean['non_mean'].mean()) / se_info
p_value_info = 2 * (1 - stats.norm.cdf(abs(z_score_info)))

print(f"Z-Score (Info vs Control, non_mean): {z_score_info}, P-Value: {p_value_info}")

# Z-test: Compare non_mean between pressure and control groups
sample_mean_pressure = data_male_pressure_non_mean['non_mean'].mean()
se_pressure = data_male_pressure_non_mean['non_mean'].std() / np.sqrt(len(data_male_pressure_non_mean))

z_score_pressure = (sample_mean_pressure - data_male_control_non_mean['non_mean'].mean()) / se_pressure
p_value_pressure = 2 * (1 - stats.norm.cdf(abs(z_score_pressure)))

print(f"Z-Score (Pressure vs Control, non_mean): {z_score_pressure}, P-Value: {p_value_pressure}")

# Z-test for pressure group comparing non_mean to con_mean
sample_mean_pressure = data_male_pressure_non_mean['non_mean'].mean()
se_pressure = data_male_pressure_non_mean['non_mean'].std() / np.sqrt(len(data_male_pressure_non_mean))

# Compare pressure group 'non_mean' to control group's 'con_mean'
z_score_con_pressure = (sample_mean_pressure - data_male_control_con_mean['con_mean'].mean()) / se_pressure
p_value_con_pressure = 2 * (1 - stats.norm.cdf(abs(z_score_con_pressure)))

print(f"Z-Score (Pressure non_mean vs Control con_mean): {z_score_con_pressure}, P-Value: {p_value_con_pressure}")


#repeat for unclear consent scenarios
import numpy as np
from scipy import stats

# Subset the data for males by group number (control, info, pressure)
data_male_control = data_male[data_male['group_number'] == 'control']
data_male_info = data_male[data_male['group_number'] == 'info']
data_male_pressure = data_male[data_male['group_number'] == 'pressure']

# Filter out rows with missing values for unc_max
data_male_control_unc_max = data_male_control.dropna(subset=['unc_max'])
data_male_info_unc_max = data_male_info.dropna(subset=['unc_max'])
data_male_pressure_unc_max = data_male_pressure.dropna(subset=['unc_max'])

# Print the means of unc_max for control, info, and pressure groups
print(f"Mean (Control, unc_max): {data_male_control_unc_max['unc_max'].mean()}")
print(f"Mean (Info, unc_max): {data_male_info_unc_max['unc_max'].mean()}")
print(f"Mean (Pressure, unc_max): {data_male_pressure_unc_max['unc_max'].mean()}")

# Z-test: Compare unc_max between info and control groups
sample_mean_info = data_male_info_unc_max['unc_max'].mean()
se_info = data_male_info_unc_max['unc_max'].std() / np.sqrt(len(data_male_info_unc_max))

z_score_info = (sample_mean_info - data_male_control_unc_max['unc_max'].mean()) / se_info
p_value_info = 2 * (1 - stats.norm.cdf(abs(z_score_info)))

print(f"Z-Score (Info vs Control, unc_max): {z_score_info}, P-Value: {p_value_info}")

# Z-test: Compare unc_max between pressure and control groups
sample_mean_pressure = data_male_pressure_unc_max['unc_max'].mean()
se_pressure = data_male_pressure_unc_max['unc_max'].std() / np.sqrt(len(data_male_pressure_unc_max))

z_score_pressure = (sample_mean_pressure - data_male_control_unc_max['unc_max'].mean()) / se_pressure
p_value_pressure = 2 * (1 - stats.norm.cdf(abs(z_score_pressure)))

print(f"Z-Score (Pressure vs Control, unc_max): {z_score_pressure}, P-Value: {p_value_pressure}")


In [ ]:
#repeating analysis for women

import numpy as np
from scipy import stats

# Subset the data for females by group number (control, info, pressure)
data_female_control = data_female[data_female['group_number'] == 'control']
data_female_info = data_female[data_female['group_number'] == 'info']
data_female_pressure = data_female[data_female['group_number'] == 'pressure']

# Function to calculate z-scores and p-values
def calculate_z_test(info_mean, control_mean, se):
    z_score = (info_mean - control_mean) / se
    p_value = 2 * (1 - stats.norm.cdf(abs(z_score)))
    return z_score, p_value

### 1. Agg Mean Analysis for Females

# Filter out rows with missing values for agg_mean
data_female_control_agg = data_female_control.dropna(subset=['agg_mean'])
data_female_info_agg = data_female_info.dropna(subset=['agg_mean'])
data_female_pressure_agg = data_female_pressure.dropna(subset=['agg_mean'])

# Print the means of agg_mean for control, info, and pressure groups
print(f"Mean (Control, agg_mean): {data_female_control_agg['agg_mean'].mean()}")
print(f"Mean (Info, agg_mean): {data_female_info_agg['agg_mean'].mean()}")
print(f"Mean (Pressure, agg_mean): {data_female_pressure_agg['agg_mean'].mean()}")

# Z-test: Compare agg_mean between info and control groups
se_info = data_female_info_agg['agg_mean'].std() / np.sqrt(len(data_female_info_agg))
z_score_info, p_value_info = calculate_z_test(data_female_info_agg['agg_mean'].mean(), data_female_control_agg['agg_mean'].mean(), se_info)
print(f"Z-Score (Info vs Control, agg_mean): {z_score_info}, P-Value: {p_value_info}")

# Z-test: Compare agg_mean between pressure and control groups
se_pressure = data_female_pressure_agg['agg_mean'].std() / np.sqrt(len(data_female_pressure_agg))
z_score_pressure, p_value_pressure = calculate_z_test(data_female_pressure_agg['agg_mean'].mean(), data_female_control_agg['agg_mean'].mean(), se_pressure)
print(f"Z-Score (Pressure vs Control, agg_mean): {z_score_pressure}, P-Value: {p_value_pressure}")

### 2. Con Mean Analysis for Females

# Filter out rows with missing values for con_mean
data_female_control_con = data_female_control.dropna(subset=['con_mean'])
data_female_info_con = data_female_info.dropna(subset=['con_mean'])
data_female_pressure_con = data_female_pressure.dropna(subset=['con_mean'])

# Print the means of con_mean for control, info, and pressure groups
print(f"Mean (Control, con_mean): {data_female_control_con['con_mean'].mean()}")
print(f"Mean (Info, con_mean): {data_female_info_con['con_mean'].mean()}")
print(f"Mean (Pressure, con_mean): {data_female_pressure_con['con_mean'].mean()}")

# Z-test: Compare con_mean between info and control groups
se_info_con = data_female_info_con['con_mean'].std() / np.sqrt(len(data_female_info_con))
z_score_info_con, p_value_info_con = calculate_z_test(data_female_info_con['con_mean'].mean(), data_female_control_con['con_mean'].mean(), se_info_con)
print(f"Z-Score (Info vs Control, con_mean): {z_score_info_con}, P-Value: {p_value_info_con}")

# Z-test: Compare con_mean between pressure and control groups
se_pressure_con = data_female_pressure_con['con_mean'].std() / np.sqrt(len(data_female_pressure_con))
z_score_pressure_con, p_value_pressure_con = calculate_z_test(data_female_pressure_con['con_mean'].mean(), data_female_control_con['con_mean'].mean(), se_pressure_con)
print(f"Z-Score (Pressure vs Control, con_mean): {z_score_pressure_con}, P-Value: {p_value_pressure_con}")

### 3. Non Mean Analysis for Females

# Filter out rows with missing values for non_mean
data_female_control_non = data_female_control.dropna(subset=['non_mean'])
data_female_info_non = data_female_info.dropna(subset=['non_mean'])
data_female_pressure_non = data_female_pressure.dropna(subset=['non_mean'])

# Print the means of non_mean for control, info, and pressure groups
print(f"Mean (Control, non_mean): {data_female_control_non['non_mean'].mean()}")
print(f"Mean (Info, non_mean): {data_female_info_non['non_mean'].mean()}")
print(f"Mean (Pressure, non_mean): {data_female_pressure_non['non_mean'].mean()}")

# Z-test: Compare non_mean between info and control groups
se_info_non = data_female_info_non['non_mean'].std() / np.sqrt(len(data_female_info_non))
z_score_info_non, p_value_info_non = calculate_z_test(data_female_info_non['non_mean'].mean(), data_female_control_non['non_mean'].mean(), se_info_non)
print(f"Z-Score (Info vs Control, non_mean): {z_score_info_non}, P-Value: {p_value_info_non}")

# Z-test: Compare non_mean between pressure and control groups
se_pressure_non = data_female_pressure_non['non_mean'].std() / np.sqrt(len(data_female_pressure_non))
z_score_pressure_non, p_value_pressure_non = calculate_z_test(data_female_pressure_non['non_mean'].mean(), data_female_control_non['non_mean'].mean(), se_pressure_non)
print(f"Z-Score (Pressure vs Control, non_mean): {z_score_pressure_non}, P-Value: {p_value_pressure_non}")

### 4. Unclear Mean Analysis for Females

# Filter out rows with missing values for unc_max
data_female_control_unc_max = data_female_control.dropna(subset=['unc_max'])
data_female_info_unc_max = data_female_info.dropna(subset=['unc_max'])
data_female_pressure_unc_max = data_female_pressure.dropna(subset=['unc_max'])

# Print the means of unc_max for control, info, and pressure groups
print(f"Mean (Control, unc_max): {data_female_control_unc_max['unc_max'].mean()}")
print(f"Mean (Info, unc_max): {data_female_info_unc_max['unc_max'].mean()}")
print(f"Mean (Pressure, unc_max): {data_female_pressure_unc_max['unc_max'].mean()}")

# Z-test: Compare unc_max between info and control groups
se_info_unc_max = data_female_info_unc_max['unc_max'].std() / np.sqrt(len(data_female_info_unc_max))
z_score_info_unc_max, p_value_info_unc_max = calculate_z_test(data_female_info_unc_max['unc_max'].mean(), data_female_control_unc_max['unc_max'].mean(), se_info_unc_max)
print(f"Z-Score (Info vs Control, unc_max): {z_score_info_unc_max}, P-Value: {p_value_info_unc_max}")

# Z-test: Compare unc_max between pressure and control groups
se_pressure_unc_max = data_female_pressure_unc_max['unc_max'].std() / np.sqrt(len(data_female_pressure_unc_max))
z_score_pressure_unc_max, p_value_pressure_unc_max = calculate_z_test(data_female_pressure_unc_max['unc_max'].mean(), data_female_control_unc_max['unc_max'].mean(), se_pressure_unc_max)
print(f"Z-Score (Pressure vs Control, unc_max): {z_score_pressure_unc_max}, P-Value: {p_value_pressure_unc_max}")


In [ ]:
#separate analysis based on how frequency participant consumes pornography outside the experiment

import numpy as np
from scipy import stats
import pandas as pd

# 1. Split into low and high frequency for 'freq_porn'
data_freq = data.dropna(subset=['freq_porn'])

# Split into low and high frequency based on 'freq_porn'
data_low_freq = data_freq[data_freq['freq_porn'] <= 2].copy()
data_high_freq = data_freq[data_freq['freq_porn'] >= 3].copy()

# Print the low and high frequency dataframes
print("Low Frequency Data:\n", data_low_freq.head())
print("High Frequency Data:\n", data_high_freq.head())

### Z-Test Function ###
def calculate_z_test(group1_mean, group2_mean, se):
    z_score = (group1_mean - group2_mean) / se
    p_value = 2 * (1 - stats.norm.cdf(abs(z_score)))
    return z_score, p_value

### 2. Mean comparison for agg_mean (low freq_porn only)

# Subset data by group_number for low freq_porn
data_low_control = data_low_freq[data_low_freq['group_number'] == 'control'].dropna(subset=['agg_mean'])
data_low_info = data_low_freq[data_low_freq['group_number'] == 'info'].dropna(subset=['agg_mean'])
data_low_pressure = data_low_freq[data_low_freq['group_number'] == 'pressure'].dropna(subset=['agg_mean'])

# Print means for agg_mean
print(f"Mean (Control, agg_mean): {data_low_control['agg_mean'].mean()}")
print(f"Mean (Info, agg_mean): {data_low_info['agg_mean'].mean()}")
print(f"Mean (Pressure, agg_mean): {data_low_pressure['agg_mean'].mean()}")

# Z-test between info and control groups
se_info = data_low_info['agg_mean'].std() / np.sqrt(len(data_low_info))
z_score_info, p_value_info = calculate_z_test(data_low_info['agg_mean'].mean(), data_low_control['agg_mean'].mean(), se_info)
print(f"Z-Score (Info vs Control, agg_mean): {z_score_info}, P-Value: {p_value_info}")

# Z-test between pressure and control groups
se_pressure = data_low_pressure['agg_mean'].std() / np.sqrt(len(data_low_pressure))
z_score_pressure, p_value_pressure = calculate_z_test(data_low_pressure['agg_mean'].mean(), data_low_control['agg_mean'].mean(), se_pressure)
print(f"Z-Score (Pressure vs Control, agg_mean): {z_score_pressure}, P-Value: {p_value_pressure}")

### 3. Mean comparison for unc_max (low freq_porn only)

# Subset data for unc_max
data_low_control_unc = data_low_control.dropna(subset=['unc_max'])
data_low_info_unc = data_low_info.dropna(subset=['unc_max'])
data_low_pressure_unc = data_low_pressure.dropna(subset=['unc_max'])

# Print means for unc_max
print(f"Mean (Control, unc_max): {data_low_control_unc['unc_max'].mean()}")
print(f"Mean (Info, unc_max): {data_low_info_unc['unc_max'].mean()}")
print(f"Mean (Pressure, unc_max): {data_low_pressure_unc['unc_max'].mean()}")

# Z-test for unc_max between info and control groups
se_info_unc = data_low_info_unc['unc_max'].std() / np.sqrt(len(data_low_info_unc))
z_score_info_unc, p_value_info_unc = calculate_z_test(data_low_info_unc['unc_max'].mean(), data_low_control_unc['unc_max'].mean(), se_info_unc)
print(f"Z-Score (Info vs Control, unc_max): {z_score_info_unc}, P-Value: {p_value_info_unc}")

# Z-test for unc_max between pressure and control groups
se_pressure_unc = data_low_pressure_unc['unc_max'].std() / np.sqrt(len(data_low_pressure_unc))
z_score_pressure_unc, p_value_pressure_unc = calculate_z_test(data_low_pressure_unc['unc_max'].mean(), data_low_control_unc['unc_max'].mean(), se_pressure_unc)
print(f"Z-Score (Pressure vs Control, unc_max): {z_score_pressure_unc}, P-Value: {p_value_pressure_unc}")

### 4. Mean comparison for agg_mean (high freq_porn only)

# Subset data by group_number for high frequency
data_high_control = data_high_freq[data_high_freq['group_number'] == 'control'].dropna(subset=['agg_mean'])
data_high_info = data_high_freq[data_high_freq['group_number'] == 'info'].dropna(subset=['agg_mean'])
data_high_pressure = data_high_freq[data_high_freq['group_number'] == 'pressure'].dropna(subset=['agg_mean'])

# Print means for agg_mean
print(f"Mean (Control, agg_mean): {data_high_control['agg_mean'].mean()}")
print(f"Mean (Info, agg_mean): {data_high_info['agg_mean'].mean()}")
print(f"Mean (Pressure, agg_mean): {data_high_pressure['agg_mean'].mean()}")

# Z-test for agg_mean between info and control groups
se_info_high = data_high_info['agg_mean'].std() / np.sqrt(len(data_high_info))
z_score_info_high, p_value_info_high = calculate_z_test(data_high_info['agg_mean'].mean(), data_high_control['agg_mean'].mean(), se_info_high)
print(f"Z-Score (Info vs Control, agg_mean): {z_score_info_high}, P-Value: {p_value_info_high}")

# Z-test for agg_mean between pressure and control groups
se_pressure_high = data_high_pressure['agg_mean'].std() / np.sqrt(len(data_high_pressure))
z_score_pressure_high, p_value_pressure_high = calculate_z_test(data_high_pressure['agg_mean'].mean(), data_high_control['agg_mean'].mean(), se_pressure_high)
print(f"Z-Score (Pressure vs Control, agg_mean): {z_score_pressure_high}, P-Value: {p_value_pressure_high}")

### 5. Z-tests for con_mean and non_mean (high freq_porn only)

# Subset data for con_mean and non_mean (high frequency)
data_high_control_con = data_high_control.dropna(subset=['con_mean'])
data_high_info_con = data_high_info.dropna(subset=['con_mean'])
data_high_pressure_con = data_high_pressure.dropna(subset=['con_mean'])

# Z-test for con_mean between info and control groups
se_info_con_high = data_high_info_con['con_mean'].std() / np.sqrt(len(data_high_info_con))
z_score_info_con_high, p_value_info_con_high = calculate_z_test(data_high_info_con['con_mean'].mean(), data_high_control_con['con_mean'].mean(), se_info_con_high)
print(f"Z-Score (Info vs Control, con_mean): {z_score_info_con_high}, P-Value: {p_value_info_con_high}")

# Z-test for con_mean between pressure and control groups
se_pressure_con_high = data_high_pressure_con['con_mean'].std() / np.sqrt(len(data_high_pressure_con))
z_score_pressure_con_high, p_value_pressure_con_high = calculate_z_test(data_high_pressure_con['con_mean'].mean(), data_high_control_con['con_mean'].mean(), se_pressure_con_high)
print(f"Z-Score (Pressure vs Control, con_mean): {z_score_pressure_con_high}, P-Value: {p_value_pressure_con_high}")

# Repeat similar Z-tests for non_mean and unc_max
data_high_control_non = data_high_control.dropna(subset=['non_mean'])
data_high_info_non = data_high_info.dropna(subset=['non_mean'])
data_high_pressure_non = data_high_pressure.dropna(subset=['non_mean'])

se_info_non_high = data_high_info_non['non_mean'].std() / np.sqrt(len(data_high_info_non))
z_score_info_non_high, p_value_info_non_high = calculate_z_test(data_high_info_non['non_mean'].mean(), data_high_control_non['non_mean'].mean(), se_info_non_high)
print(f"Z-Score (Info vs Control, non_mean): {z_score_info_non_high}, P-Value: {p_value_info_non_high}")

# Z-test for non_mean between pressure and control groups
se_pressure_non_high = data_high_pressure_non['non_mean'].std() / np.sqrt(len(data_high_pressure_non))
z_score_pressure_non_high, p_value_pressure_non_high = calculate_z_test(data_high_pressure_non['non_mean'].mean(), data_high_control_non['non_mean'].mean(), se_pressure_non_high)
print(f"Z-Score (Pressure vs Control, non_mean): {z_score_pressure_non_high}, P-Value: {p_value_pressure_non_high}")

### 6. Linear Regression Comparisons for agg_mean and freq_porn

# Linear regression for control group
regression_control = sm.OLS.from_formula('agg_mean ~ gender_identity + freq_porn', data=data_control).fit()
print(regression_control.summary())

# Linear regression for info group
regression_info = sm.OLS.from_formula('agg_mean ~ gender_identity + freq_porn', data=data_info).fit()
print(regression_info.summary())

# Linear regression for pressure group
regression_pressure = sm.OLS.from_formula('agg_mean ~ gender_identity + freq_porn', data=data_pressure).fit()
print(regression_pressure.summary())


In [ ]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
import statsmodels.api as sm
from scipy import stats

# 1. Linear regression to test if gender differences are fully explained by freq_porn
model = ols('agg_mean ~ treatment + gender_identity + freq_porn', data=data).fit()
print(model.summary())

# Test the significance of the gender coefficient after controlling for freq_porn
model_controlled = ols('agg_mean ~ treatment + freq_porn + gender_identity:freq_porn', data=data).fit()
anova_results = sm.stats.anova_lm(model, model_controlled)
print(anova_results)

# 2. Linear regressions for male and female participants separately
# For males
print(ols('agg_mean ~ freq_porn', data=data_male).fit().summary())

# For females
print(ols('agg_mean ~ freq_porn', data=data_female).fit().summary())

# 3. Double difference model: Interaction between guilt and treatment, for males and females

# Subset data for males and females
data_male = data[data['gender_identity'] == 'male']
data_female = data[data['gender_identity'] == 'female']

# Create new 'guilt' and 'treat_yes' columns based on thresholds
data_male['guilt'] = np.where(data_male['guilt_feelings'] <= 2, 0, 1)
data_female['guilt'] = np.where(data_female['guilt_feelings'] <= 2, 0, 1)
data_male['treat_yes'] = np.where((data_male['group_number'] == 'info') | (data_male['group_number'] == 'pressure'), 1, 0)
data_female['treat_yes'] = np.where((data_female['group_number'] == 'info') | (data_female['group_number'] == 'pressure'), 1, 0)

# Run linear regression models with interaction terms
print(ols('agg_mean ~ treat_yes + guilt + treat_yes*guilt', data=data_male).fit().summary())
print(ols('agg_mean ~ treat_yes + guilt + treat_yes*guilt', data=data_female).fit().summary())

# 4. Interaction between guilt and group_number for men
print(ols('agg_mean ~ guilt * group_number', data=data_male).fit().summary())

# 5. Triple difference model to test interactions between guilt, treatment, and gender
data['guilt'] = np.where(data['guilt_feelings'] <= 2, 0, 1)
data['treat_yes'] = np.where((data['group_number'] == 'info') | (data['group_number'] == 'pressure'), 1, 0)

# Linear regression model with triple interaction terms
print(ols('agg_mean ~ guilt + treat_yes + guilt*treat_yes + gender_identity + gender_identity*treat_yes + treat_yes*guilt*gender_identity', data=data).fit().summary())

# 6. Correlation between freq_porn and comfort with non-consensual scenarios
non_mean_correlation = data[['non_mean', 'freq_porn']].corr().loc['non_mean', 'freq_porn']
unc_max_correlation = data[['unc_max', 'freq_porn']].corr().loc['unc_max', 'freq_porn']

print(f"Correlation between non_mean and freq_porn: {non_mean_correlation}")
print(f"Correlation between unc_max and freq_porn: {unc_max_correlation}")
